# Minimum Weighted Maximum Matching

問題：  
会社に属する新卒社員N人に対し，それぞれの希望順位に基づき2人1組のマッチングを行いたい(グループ数：N/2)．  

解法：  
仮に6人(A,B,C,D,E,F)がいるとした時，何かしらの情報を基にそれぞれの希望順位を書き出してもらう(今回は3位まで)．  
そして，希望順位を基にそれぞれに点数を与える(1位:1pt,2位:2pt,3位:3pt,4位~:10pt)  
- A, 1位:C 2位:D 3位:F
- B, 1位:E 2位:F 3位:C
- C, 1位:B 2位:E 3位:D
- D, 1位:F 2位:A 3位:E
- E, 1位:A 2位:F 3位:C
- F, 1位:B 2位:D 3位:A 

続いて，上記の希望順位に基づき，[重み付き無向グラフ](https://algo-logic.info/graph/)を構築する．  
この時の重みは，それぞれの希望順位による点数の総和とする．  
例えば，A→Cは1位なので1pt，C→Aは圏外なので10pt，したがってA-C間の辺の重みは11ptである．  
そうすると，これらは次のグラフのように表現することが可能である．  
<img src="./img/graph.jpg" width="50%">

したがって，全員の希望をいい感じに考慮しつつ問題で挙げたマッチングを行うには  
- 必ず誰かしらとペアになる(最大マッチング)  
- その際に辺の重みの総和を最小にする(最小費用)  

を達成する必要がある(本当はばらつきも考慮すべきだが，今回は簡略化のため除く)．

そのため今回は，『最小重み最大マッチング(Minimum Weighted Maximum Matching; mwmm)』を用いることで，
これらの解決を試みる．  
以下にソースコードを記述する．

## 参考資料

 - [マッチング問題とその変形に対するアルゴリズム](https://scmopt.github.io/opt100/30matching.html)  
- [python: NetworkX による無向/有向グラフの読み込みと可視化](https://www.letsopt.com/entry/2019/10/22/214535)  

# ソースコード